In [ ]:
import requests
import pandas as pd
import json
from io import StringIO

def fetch_oecd_data(metric):
    url = f"https://sdmx.oecd.org/public/rest/data/{metric}/all?startPeriod=2000&endPeriod=2025&dimensionAtObservation=AllDimensions&format=csvfilewithlabels"
    response = requests.get(url)
    print(response.status_code)
    # digesting the csv text into a pandas df
    df = pd.read_csv(StringIO(response.text)) #stringio makes it "appear" like a csv to pandas
    #df = df[['TIME_PERIOD', 'Reference area', 'REF_AREA', 'Measure','OBS_VALUE']] # selecting columns to keep
    return df

TradeUnion_code = 'OECD.ELS.SAE,DSD_TUD_CBC@DF_TUD,'
CorpTax_code = 'OECD.CTP.TPS,DSD_TAX_CIT@DF_CIT,1.0/.A..ST..S13+S1311+S13M..'
GovSpend_code = 'OECD.SDD.NAD,DSD_NASEC10@DF_TABLE11,/A..S13...OTE.....V..'

df = fetch_oecd_data(GovSpend_code)
print(df.shape)
#df.to_csv('GovSpend.csv', index=False)

200
(63750, 50)


In [8]:
def split_gov_expenditure(df, sectors):
    df_dict = {}
    for sector in sectors:
        df_dict[sector] = df.loc[df['Expenditure'] == sector][['Reference area', 'TIME_PERIOD', 'OBS_VALUE', 'UNIT_MULT', 'CURRENCY']]
    return df_dict

split_gov_expenditure(df, ['Education', 'Health', 'Housing', 'Community development'])

{'Education':       Reference area  TIME_PERIOD  OBS_VALUE  UNIT_MULT CURRENCY
 1575       Australia         2023   151648.0          6      AUD
 1576       Australia         2022   141535.0          6      AUD
 1577       Australia         2021   133905.0          6      AUD
 1578       Australia         2020   126629.0          6      AUD
 1579       Australia         2019   125910.0          6      AUD
 ...              ...          ...        ...        ...      ...
 63132         Norway         2002    90015.0          6      NOK
 63133         Norway         2003    99262.0          6      NOK
 63134         Norway         2004   100813.0          6      NOK
 63135         Norway         2005   103663.0          6      NOK
 63136         Norway         2006   109005.0          6      NOK
 
 [865 rows x 5 columns],
 'Health':       Reference area  TIME_PERIOD  OBS_VALUE  UNIT_MULT CURRENCY
 1607       Australia         2004    59114.0          6      AUD
 1608       Australia     

In [19]:
url = 'https://api.worldbank.org/v2/country/all/indicator/PA.NUS.FCRF?format=json&date=2000:2023&per_page=1000'
response = requests.get(url).json()
response

[{'page': 1,
  'pages': 7,
  'per_page': 1000,
  'total': 6384,
  'sourceid': '2',
  'lastupdated': '2025-04-15'},
 [{'indicator': {'id': 'PA.NUS.FCRF',
    'value': 'Official exchange rate (LCU per US$, period average)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2023',
   'value': None,
   'unit': '',
   'obs_status': '',
   'decimal': 2},
  {'indicator': {'id': 'PA.NUS.FCRF',
    'value': 'Official exchange rate (LCU per US$, period average)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2022',
   'value': None,
   'unit': '',
   'obs_status': '',
   'decimal': 2},
  {'indicator': {'id': 'PA.NUS.FCRF',
    'value': 'Official exchange rate (LCU per US$, period average)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2021',
   'value': None,
   'unit': '',
   'obs_status': '',
   'decimal': 2